In [1]:
# Importing the Libraries
import pyspark
from pyspark.sql import SparkSession

# Function to Prompt user to enter Zip code and verify format
def get_zipcode():
    while True:
        zip_code = input("Enter zip code (for example: 01824)").strip()

        # Verify zip code has 5 digits
        if zip_code.isdigit() and len(zip_code) == 5:
            return zip_code
        else:
            print("Invalid zip code entered. Please enter a 5-digit numeric zip code.")

def get_month_year():
    while True:
        month = input("Enter Month (1-12): ").strip()
        year = input("Enter Year (for example: 2024): ").strip()
        
        # Verify month is between 1 & 12 and year has 4 digits
        if (month.isdigit() and 1<= int(month) <=12) and (year.isdigit() and len(year) == 4):
            return int(month), int(year)
        else:
            print("Invalid input. Please enter a valid month (1-12) and 4-digit numeric year.")

def query_transactions(zip_code, month, year):
    #jdbc parameters
    jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"
    # dbtable_query = f'''select timeid as Date, TRANSACTION_TYPE as Type, 
    #                     TRANSACTION_VALUE as Amount, cust.FIRST_NAME as "First Name", cust.LAST_NAME as "Last Name"
    #                 from cdw_sapp_credit_card cc
    #                 join cdw_sapp_customer cust on cust.CREDIT_CARD_NO = cc.CUST_CC_NO
    #                 where cc.month = {month} and cc.year = {year} and cust_zip = {zip_code}
    #                 order by day desc;'''
    dbtable_query = f'''(SELECT timeid AS Date, TRANSACTION_TYPE AS Type, 
                    TRANSACTION_VALUE AS Amount, cust.FIRST_NAME AS "First Name", cust.LAST_NAME AS "Last Name"
                    FROM cdw_sapp_credit_card cc
                    JOIN cdw_sapp_customer cust ON cust.CREDIT_CARD_NO = cc.CUST_CC_NO
                    WHERE cc.month = {month} AND cc.year = {year} AND cust_zip = {zip_code}
                    ORDER BY day DESC) AS filtered_transactions'''
    # dbtable_query = f"(SELECT * FROM CDW_SAPP_CREDIT_CARD WHERE CUST_ZIP = '{zip_code}' AND LAST_UPDATED LIKE '{year}-{month:02d}%') AS filtered_transactions"
    user = "root"
    password = "password"

     # Creating Spark Session
    spark = SparkSession.builder.appName('Query Transactions').getOrCreate()

    # Create a DataFrame reader using the JDBC connection and query
    try:
        df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", dbtable_query) \
            .option("user", user) \
            .option("password", password) \
            .load()

        return df

    except Exception as e:
        print(f"Error querying data: {str(e)}")
        return None

def get_transaction_detail():
    zip_code = get_zipcode()
    month, year = get_month_year()
    df = query_transactions(zip_code, month, year)

    if df is not None:
        df.show(df.count(), truncate=False)
    else:
        print("No data retrieved or error occurred.")

   
if __name__ == "__main__":
    get_transaction_detail()
    #print(f"Valid zip code entered: {zip_code}")


+--------+-------------+------+----------+---------+
|Date    |Type         |Amount|First Name|Last Name|
+--------+-------------+------+----------+---------+
|20180528|Gas          |57.66 |Melissa   |Hagan    |
|20180526|Gas          |57.04 |Blair     |Ware     |
|20180526|Grocery      |58.88 |Alec      |Hooper   |
|20180523|Grocery      |27.63 |Fermin    |Carson   |
|20180523|Healthcare   |22.42 |Blair     |Ware     |
|20180522|Test         |24.53 |Alec      |Hooper   |
|20180521|Education    |47.45 |Kraig     |Todd     |
|20180521|Bills        |64.77 |Fermin    |Carson   |
|20180519|Gas          |77.09 |Melissa   |Hagan    |
|20180518|Bills        |9.27  |Omer      |Dickinson|
|20180518|Test         |44.25 |Blair     |Ware     |
|20180518|Grocery      |64.83 |Judson    |Macias   |
|20180517|Entertainment|61.84 |Isabel    |Cramer   |
|20180517|Healthcare   |27.69 |Alec      |Hooper   |
|20180516|Gas          |55.02 |Blair     |Ware     |
|20180516|Grocery      |60.78 |Fermin    |Cars